In [1]:
print("teste")

teste


In [2]:
import yfinance as yf

In [4]:
data = yf.download("AAPL", start="2020-01-01", end="2020-12-31")

[*********************100%%**********************]  1 of 1 completed


In [5]:
p = 3

data["Return"] = data["Adj Close"].pct_change(p)
data["Target"] = data["Return"].shift(-p)
data["Average"] = data["Return"].rolling(52).mean()
data["RSL"] = (data["Adj Close"] / data["Average"]-1)

limite_superior = 0.01
limite_inferior = -0.01

In [8]:
def classify_change(change):
    if change > limite_superior:
        return "Aumento"
    elif change < limite_inferior:
        return "Diminuição"
    else:
        return "Estável"

In [10]:
year_train = 2020
year_test = 2021

start_train = f"{year_train}-01-01"
end_train = f"{year_train}-12-31"

start_test = f"{year_test}-01-01"
end_test = f"{year_test}-12-31"

data_train = data[start_train:end_train]
data_test = data[start_test:end_test]

In [11]:
data_train["Class"] = data_train["Return"].apply(classify_change)

C:\Users\maste\AppData\Local\Temp\ipykernel_19064\3851964912.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_train["Class"] = data_train["Return"].apply(classify_change)


In [12]:
estados = ["Aumento", "Diminuição", "Estável"]
transicao = {estado : {e : 0 for e in estados} for estado in estados}

In [13]:
for i in range(1, len(data_train)):
    estado_atual = data_train["Class"].iloc[i-1]
    estado_futuro = data_train["Class"].iloc[i]
    transicao[estado_atual][estado_futuro] += 1

In [14]:
for estado_atual, transicoes in transicao.items():
    total = sum(transicoes.values())

    for estado in estados:
        transicao[estado_atual][estado] /= total

In [16]:
for estado_atual, transicoes in transicao.items():
    print(f"Estado atual: {estado_atual}")
    for estado, probabilidade in transicoes.items():
        print(f"Estado futuro: {estado} - Probabilidade: {probabilidade: .2f}")
    print()

Estado atual: Aumento
Estado futuro: Aumento - Probabilidade:  0.71
Estado futuro: Diminuição - Probabilidade:  0.11
Estado futuro: Estável - Probabilidade:  0.18

Estado atual: Diminuição
Estado futuro: Aumento - Probabilidade:  0.21
Estado futuro: Diminuição - Probabilidade:  0.59
Estado futuro: Estável - Probabilidade:  0.20

Estado atual: Estável
Estado futuro: Aumento - Probabilidade:  0.36
Estado futuro: Diminuição - Probabilidade:  0.27
Estado futuro: Estável - Probabilidade:  0.37



In [21]:
data["Class"] = data["Return"].apply(classify_change)

In [17]:
def get_transicao_probabilidade(row, transicao):
    current_class = row["Class"]
    previous_class = row["Previous_Class"]
    return transicao.get(current_class, {}).get(previous_class, 0)

In [22]:
data["Previous_Class"] = data["Class"].shift(1)

In [23]:
data["M_prob"] = data.apply(lambda row: get_transicao_probabilidade(row, transicao), axis=1)

In [24]:
data

,Open,High,Low,Close,Adj Close,Volume,Return,Target,Average,RSL,Class,Previous_Class,M_prob
Date,,,,,,,,,,,,,
2020-01-02,74.059998,75.150002,73.797501,75.087502,73.059418,135480400,NaN,-0.006526,NaN,NaN,Estável,None,0.000000
2020-01-03,74.287498,75.144997,74.125000,74.357498,72.349136,146322800,NaN,0.019366,NaN,NaN,Estável,Estável,0.372881
2020-01-06,73.447502,74.989998,73.187500,74.949997,72.925636,118387200,NaN,0.032788,NaN,NaN,Estável,Estável,0.372881
2020-01-07,74.959999,75.224998,74.370003,74.597504,72.582657,108872000,-0.006526,0.040015,NaN,NaN,Estável,Estável,0.372881
2020-01-08,74.290001,76.110001,74.290001,75.797501,73.750252,132079200,0.019366,0.045417,NaN,NaN,Aumento,Estável,0.181818
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-12-23,132.160004,132.429993,130.779999,130.960007,128.523270,88223700,0.033949,0.029856,0.007597,16917.206461,Aumento,Aumento,0.710744
2020-12-24,131.320007,133.460007,131.100006,131.970001,129.514481,54930100,0.029166,0.013260,0.006600,19621.850180,Aumento,Aumento,0.710744
2020-12-28,133.990005,137.339996,133.509995,136.690002,134.146652,124486200,0.036472,NaN,0.006276,21372.764372,Aumento,Aumento,0.710744
